<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Scope-of-the-Project" data-toc-modified-id="Scope-of-the-Project-0.1">Scope of the Project</a></span></li><li><span><a href="#Changes-to-the-PyMC3-Code-Base" data-toc-modified-id="Changes-to-the-PyMC3-Code-Base-0.2">Changes to the PyMC3 Code Base</a></span><ul class="toc-item"><li><span><a href="#A-new-module:-'Backends_Symbolic'" data-toc-modified-id="A-new-module:-'Backends_Symbolic'-0.2.1">A new module: 'Backends_Symbolic'</a></span></li><li><span><a href="#Changes-needed-for-model.py" data-toc-modified-id="Changes-needed-for-model.py-0.2.2">Changes needed for model.py</a></span><ul class="toc-item"><li><span><a href="#TensorVariables" data-toc-modified-id="TensorVariables-0.2.2.1">TensorVariables</a></span></li><li><span><a href="#TensorVariable-related-functions" data-toc-modified-id="TensorVariable-related-functions-0.2.2.2">TensorVariable-related functions</a></span></li></ul></li><li><span><a href="#Changes-needed-for-distribution.py" data-toc-modified-id="Changes-needed-for-distribution.py-0.2.3">Changes needed for distribution.py</a></span><ul class="toc-item"><li><span><a href="#Getting-Values-from-Symbolic-Variables" data-toc-modified-id="Getting-Values-from-Symbolic-Variables-0.2.3.1">Getting Values from Symbolic Variables</a></span></li><li><span><a href="#TensorFlow-Sessions" data-toc-modified-id="TensorFlow-Sessions-0.2.3.2">TensorFlow Sessions</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-0.2.3.3">Functions</a></span></li></ul></li><li><span><a href="#Changes-needed-for-continuous.py" data-toc-modified-id="Changes-needed-for-continuous.py-0.2.4">Changes needed for continuous.py</a></span><ul class="toc-item"><li><span><a href="#Basic-Math" data-toc-modified-id="Basic-Math-0.2.4.1">Basic Math</a></span></li></ul></li><li><span><a href="#Changes-needed-for-arraystep.py" data-toc-modified-id="Changes-needed-for-arraystep.py-0.2.5">Changes needed for arraystep.py</a></span></li><li><span><a href="#Changes-needed-for-metropolis.py" data-toc-modified-id="Changes-needed-for-metropolis.py-0.2.6">Changes needed for metropolis.py</a></span></li></ul></li><li><span><a href="#Test-Case:-A-Motivating-Example:-Linear-Regression" data-toc-modified-id="Test-Case:-A-Motivating-Example:-Linear-Regression-0.3">Test Case: A Motivating Example: Linear Regression</a></span><ul class="toc-item"><li><span><a href="#Running-with-original-pymc3-code-(restart-kernel)" data-toc-modified-id="Running-with-original-pymc3-code-(restart-kernel)-0.3.1">Running with original pymc3 code (restart kernel)</a></span></li><li><span><a href="#Running-with-Theano-backend-(restart-kernel-for-a-fresh-import)" data-toc-modified-id="Running-with-Theano-backend-(restart-kernel-for-a-fresh-import)-0.3.2">Running with Theano backend (restart kernel for a fresh import)</a></span></li><li><span><a href="#Running-with-TensorFlow-backend-(restart-kernel-for-a-fresh-import)" data-toc-modified-id="Running-with-TensorFlow-backend-(restart-kernel-for-a-fresh-import)-0.3.3">Running with TensorFlow backend (restart kernel for a fresh import)</a></span></li></ul></li></ul></li></ul></div>

## Scope of the Project

PyMC3 does much of its computation using [Theano](http://deeplearning.net/software/theano/). As Theano is no longer going to be maintained, the developers of PyMC3 have expressed interest in allowing for multiple symbolic backends for pymc3. This venture has been started as PyMC4 (see this [discussion thread](https://discourse.pymc.io/t/tensorflow-backend-for-pymc4/409/15)).

Within this github repo, I altered some of the PyMC3 source code so that it can run on either Theano or [TensorFlow](https://www.tensorflow.org/) backends. [Keras](https://keras.io/) is a package with similar functionality, and I used it as a model for this project (see backends [documenation](https://keras.io/backend/) or the [source code](https://github.com/keras-team/keras/tree/master/keras/backend) for Keras).

Note that the term 'symbolic backends' differs from 'backends' in PyMC3, which is used for different storage options (hdf5, ndarrays etc.) for MCMC traces. 


In [1]:
from utils import display_code


## Changes to the PyMC3 Code Base

I've only converted the code required to recreate this basic example ["A Motivating Example: Linear Regression"](http://docs.pymc.io/notebooks/getting_started#A-Motivating-Example:-Linear-Regression). However, some principles that I used to accomplish this, can hopefully be extended to the rest of the code base. 

I have a local version of the PyMC3 code base here called 'pymc3-master'. It consists of these modules: 

In [130]:
!ls ./pymc3-master/pymc3

__init__.py       distributions     memoize.py        theanof.py
__pycache__       examples          model.py          tuning
backends          exceptions.py     plots             util.py
backends_symbolic external          sampling.py       variational
blocking.py       glm               stats.py          vartypes.py
data.py           gp                step_methods
diagnostics.py    math.py           tests


In order to run the basic example, I needed to alter: 
- `__init__.py`
- `model.py`
- `distributions/distribution.py`
- `distributions/continuous.py`
- `step_methods/arraystep.py`
- `step_methods/metropolis.py`

I'll go through the changes needed for each module in turn. 

I also heavily commented code that I did not want to change, but was imported with PyMC3's initial import. For example, I only kept the `Normal` distribution in `continuous.py`, and commented all the rest out. 


### A new module: 'Backends_Symbolic'


Much of the Theano functionality is either directly imported from Theano at the top of each module, or imported from
PyMC3 modules such as `theanof.py` or `math.py`. I commented these imports and instead import all the necessary functionality from my new module called `backends_symbolic`. Here is an example, of how I comment out the imports and replace with a single import to my new backend. 


In [13]:
display_code('model.py',[8,20])

Within `symbolic_backends`, I have implementations of the same functions in two different modules, `backends_tf.py` and `backends_theano.py`.


In [9]:
!ls ./pymc3-master/pymc3/*backends_symbolic*

__init__.py        backends_tf.py     common.py
__pycache__        backends_theano.py


The default symbolic backend is Theano, but TensorFlow can be specified by setting an environment variable `os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" `. This environment variable is referenced during the import of the module, so that either the TensorFlow or Theano implementations of each function are referenced. 


In [16]:
file = 'backends_symbolic/__init__.py'
lines=[4,20]
display_code(file,lines)

### Changes needed for model.py

The essential object of PyMC3 is a model, which consists of linked random variables. The two main random variables are ObservedRV and FreeRV, which have and do not have observed data, respectively. 


#### TensorVariables

These random variable classes were originally a subclass of the `TensorVariable` in Theano, but now they differ depending on backend. In Theano, it's still a `TensorVariable`, and in TensorFlow, it's a `Variable`.  

Here is an example of the class change in `models.py`. Note that I left a `##CHANGED:` comment everywhere that I adjusted the original PyMC3 code. 

In [19]:
file = 'model.py'
lines=[1234,1239]
display_code(file,lines)

And here is the basically empty wrapper in the backends modules. 

In [28]:
file = 'backends_symbolic/backends_tf.py'
lines=[1,17]
display_code(file,lines)

One implication of this change is that using a TensorFlow backend requires an `initial_value` be specified for all variables, even free random variables. 

#### TensorVariable-related functions

There are a few other tensor variable related functions that I needed to implement for both backends. I've added doc strings, but these functions are pretty self-explanatory.

In [32]:
file = 'backends_symbolic/backends_tf.py'
lines=[25,50]
display_code(file,lines)

### Changes needed for distribution.py

Free and observed random variables in PyMC each come with `Distribution` classes. I only altered the `Normal` distribution in the `Continuous` distribution class. 

#### Getting Values from Symbolic Variables


Distributions have functions, such as sample, that require getting the actual values from symbolic variables. 

Getting values differs in Theano and TensorFlow. In Theano, if a variable has a value, it can be obtained directly. The type of variable needs to be checked beforehand, however, because the call is slightly different. 

In [47]:
file = 'backends_symbolic/backends_theano.py'
lines=[45,56]
display_code(file,lines)

In TensorFlow, a variable needs to be fed to a session and the session needs to be run in order to return a value. 


In [52]:
file = 'backends_symbolic/backends_tf.py'
lines=[54,60]
display_code(file,lines)

In TensorFlow, a variable needs to be fed to a session and the session needs to be run in order to return a value. 

#### TensorFlow Sessions

The TensorFlow backend deals with instantiating a session the first time around and retrieving it when needed

In [128]:
file = 'backends_symbolic/backends_tf.py'
lines=[80,88]
display_code(file,lines)

#### Functions

Another major change was to replace the calls to `theano.function`, with a backend-agnostic implementation. Keras' implementation helped a lot here. 

For example, `distribution.py` has a function that draws values from a distribution. It uses `_compile_theano_function`, a memoized and wrapped version of `theano.function`, which returns a compiled Theano function.  

In [67]:
file = 'distributions/distribution.py'
lines=[398,405]
display_code(file,lines)

I replaced this with a call to my backend. Like keras, I've wrapped `theano.function` into a class called `Function`. The initialization of the class upon instantiation compiles the function, and the call evaluates it. Here's the implementation in my backend.  

In [132]:
file = 'backends_symbolic/backends_theano.py'
lines=[97,135]
display_code(file,lines)

Like Keras, there is also an accompanying `function` function which instantiates the `Function` class. :)

In [133]:
file = 'backends_symbolic/backends_theano.py'
lines=[135,145]
display_code(file,lines)

This implementation allows us to use the same syntax for TensorFlow, which doesn't compile a function per se. Here, the intialization of `Function` relates the inputs, outputs, updates, and session variable. The call then evaluates the output variables in the session. 


In [135]:
file = 'backends_symbolic/backends_tf.py'
lines=[130,205]
display_code(file,lines)

This `Function` class is used for when PyMC's `Model` class compiles the likelihood function using the `logp` output variables and the various input variables.


### Changes needed for continuous.py

The `Normal` distribution class contains the parameters of the distribution. Here, for example, there is the standard deviation and mean for the normal distribution. 

In [129]:
file = 'distributions/continuous.py'
lines=[297,304]
display_code(file,lines)

It also contains the symbolic definition of the loglikelihood. These are the 'outputs' in the computation graph and with the 'inputs' will be be compiled into a Theano function or fed to a TensorFlow session. 

In [59]:
file = 'distributions/continuous.py'
lines=[322,331]
display_code(file,lines)

#### Basic Math
All of the basic mathematical functions required for specifying the likelihood have been moved to the backend. Here are some examples.

In [64]:
file = 'backends_symbolic/backends_tf.py'
lines=[220,234]
display_code(file,lines)

### Changes needed for arraystep.py

The MCMC sampling in pymc is done by taking what's called a 'step' in parameter space from the current parameter values to a new set of values. The `ArrayStepShared` class takes one step at a time for each parameter. 

Each parameter has it's own instantiation of a particular step method, for example the Metropolis-Hasting method. Therefore the `step` function printed below from `ArrayStepShared` gets called for each variable in the model. 

For now, I've hacked together a way to implement the step in both Theano and TensorFlow. The simplest solution for now was to write a conditional for the different backends. In Theano, we only pass one input variable to the step method. In TensorFlow, I'm passing all the variables. 

In [89]:
file = 'step_methods/arraystep.py'
lines=[162,184]
display_code(file,lines)

### Changes needed for metropolis.py

A Metropolis-Hasting step works as follows: 
- For a particular variable, and random step in parameter space is proposed from some proposal distribution (usually a normal distribution). 
 

In [100]:
file = 'step_methods/metropolis.py'
lines=[163,164]
display_code(file,lines)

- This proposed step `delta` is added to the original parameter value `q0`, to create `q`. 
- Then the change in the models log likelihood, `delta_logp`, is calculated.
- If the change in likelihood is greater than some threshold, the new parameter value `q` is accepted

In [108]:
file = 'step_methods/metropolis.py'
lines=[178,182]
display_code(file,lines)

I've moved the calculation of the change in log likelihood to the symbolic backend. A compiled function gets associated with the metroplis step method upon its instantiation for each variable.  


In [99]:
file = 'backends_symbolic/backends_theano.py'
lines=[312,330]
display_code(file,lines)

For TensorFlow, I've implemented it slightly differently at the moment. Instead of having a `delta_logp` function, I just call the `logp` function twice, once for the old and new values, and calculate the difference. This is slower, but was an easier first pass. 

In [118]:
file = 'step_methods/metropolis.py'
lines=[184,204]
display_code(file,lines)

Again, both of these functions rely on the `Function` class. 

In [125]:
file = 'backends_symbolic/backends_tf.py'
lines=[283,294]
display_code(file,lines)

## Test Case: A Motivating Example: Linear Regression

### Running with original pymc3 code (restart kernel)

Here I import an unaltered PyMC3 package. 

In [8]:
import sys
import numpy as np
sys.path.append('/Users/chris/anaconda/envs/env_pymc3/lib/python3.6/')
sys.path.append('/Users/chris/anaconda/envs/env_pymc3/lib/python3.6/site-packages/')
import pymc3 as pm

In [9]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [10]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [beta]
>Metropolis: [alpha]
100%|██████████| 5500/5500 [00:01<00:00, 3819.80it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.9054529
[0.9458999 2.6006136]


These are the estimated parameter values for alpha and beta. They are reasonably close to the true values of [1] and [1, 2.5]. 

### Running with Theano backend (restart kernel for a fresh import)

Here I import the local PyMC3 package that I changed. I set the backend to be Theano. 

In [1]:
import sys
import os
import numpy as np
os.environ["PYMC_SYMB_BACKEND"] = "theano" 
sys.path.append('./pymc3-master/')
import pymc3 as pm

Using Theano backend.
/Users/chris/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [beta]
>Metropolis: [alpha]
100%|██████████| 5500/5500 [00:02<00:00, 2738.67it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.9054529
[0.9458999 2.6006136]


This shows that that my code base produces the same answer as the original PyMC3 code base. 

### Running with TensorFlow backend (restart kernel for a fresh import)

After restarting the kernel, I import the local PyMC3 package again, but now I set the backend to be TensorFlow.

In [1]:
import sys
import os
import numpy as np
sys.path.append('./pymc3-master/')
os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" 
import pymc3 as pm

Using TensorFlow backend.
/Users/chris/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    
    # requires initial value now. 
    # also has to be float at the moment. 
    alpha = pm.Normal(name='alpha',initial_value=0.0, mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',initial_value=np.array([1.0,1.0],dtype='float32'), mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',initial_value=np.ones_like(Y), mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1)

print(np.mean(trace['alpha:0']))
print(np.mean(trace['beta:0'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [<tf.Variable 'alpha:0' shape=() dtype=float32_ref>]
>Metropolis: [<tf.Variable 'beta:0' shape=(2,) dtype=float32_ref>]
100%|██████████| 5500/5500 [00:12<00:00, 455.22it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.905485
[0.947812 2.62094 ]


It's a bit slower, but it works! Notice the variable names are `tf.Variables` now. 

The parameter estimates are virtually the same, but not exactly. One reason for the slight differences may be that I set the TensorFlow variables to be float32's and Theano variables default to float64's. 